# VFE

Maximize KL divergence ELBO:

\begin{align}
\mathbf{F}(q, \boldsymbol{\theta}) &= \log p(\mathbf{y} | \boldsymbol{\theta}) + \int q(f | \mathbf{y}) \log \frac{p(f | \mathbf{y})}{q(f | \mathbf{y})}\\
&= \int q(f | \mathbf{y}) \log \frac{p(\mathbf{y}, f | \boldsymbol{\theta})}{q(f | \mathbf{y})}
\end{align}

Note that we can choose the form of $q(f | \mathbf{y})$ as we like. Choose a special approximate posterior:

\begin{align}
q(f | \mathbf{y}) = p\left(f_{\neq \mathbf{\bar{X}}} | \mathbf{f}_{\mathbf{\bar{X}}}\right) q\left(\mathbf{f}_{\mathbf{\bar{X}}} | \mathbf{y} \right)
\end{align}

Maximise $\mathcal{F}$ w.r.t. $q$:

\begin{align}
q^*\left(\mathbf{f}_{\mathbf{\bar{X}}} | \mathbf{y} \right) = \text{argmax}_{q} \mathbf{F}(q, \boldsymbol{\theta})
\end{align}



## Solve for maximising the free energy

Substitute special prior in free energy bound:

\begin{align} \require{cancel}
\mathbf{F}(q, \boldsymbol{\theta}) &= \int p\lrb[f_{\neq \Xb} | \fb] q\lrb[\fb | \mathbf{y}] \log \frac{p(\mathbf{y}, f | \bs[\theta])}{p\lrb[f_{\neq \Xb} | \fb] q\lrb[\mathbf{f}_{\mathbf{\bar{X}}} | \mathbf{y}]}\\
&= \int p\lrb[f_{\neq \Xb} | \fb] q\lrb[\fb | \mathbf{y}] \log \frac{p(\mathbf{y} | \mathbf{f}, \bs[\theta])  \cancel{p\lrb[f_{\neq \Xb} | fb]}  p\lrb[\fb]}{\cancel{p\lrb[f_{\neq \Xb} | \fb]} q\left(\fb | \mathbf{y} \right)} df\\
&= \int p\lrb[\fx | \fb] q\lrb[\fb | \mathbf{y}] \log \frac{p(\mathbf{y} | \mathbf{f}, \bs[\theta]) p(\fb)}{ q\left(\mathbf{f}_{\mathbf{\bar{X}}} | \mathbf{y} \right)} d\mathbf{f}_{\mathbf{X}} d\mathbf{f}_{\mathbf{\bar{X}}}
\end{align}

Substituting $p\left(f_{\neq \mathbf{\bar{X}}} | \mathbf{f}_{\mathbf{\bar{X}}}\right) = \mathcal{N}\left(f_{\neq \mathbf{\bar{X}}}; \mathbf{K}_{\mathbf{X}\mathbf{\bar{X}}} \mathbf{K}_{\mathbf{\bar{X}}\mathbf{\bar{X}}}^{-1},  \mathbf{K}_{\mathbf{\bar{X}}\mathbf{\bar{X}}} - \mathbf{K}_{\mathbf{X}\mathbf{\bar{X}}} \mathbf{K}_{\mathbf{\bar{X}}\mathbf{\bar{X}}}^{-1} \right)$

## Observations

Tightness of bound and expressiveness of approximate posterior are different things.

$
 \def\Kxx{\mathbf{K}_{\mathbf{X}\mathbf{X}}}
 \def\Kxb{\mathbf{K}_{\mathbf{X}\mathbf{\bar{X}}}}
 \def\Kbx{\mathbf{K}_{\mathbf{\bar{X}}\mathbf{X}}}
 \def\Kbb{\mathbf{K}_{\mathbf{\bar{X}}\mathbf{\bar{X}}}}
 \def\fx{\mathbf{f}_{\mathbf{X}}}
 \def\fb{\mathbf{f}_{\mathbf{\bar{X}}}}
 \def\X{\mathbf{X}}
 \def\Xb{\mathbf{\bar{X}}}
 \def\lrb[#1]{\left(#1\right)}
 \def\bm[#1]{\mathbf{#1}}
 \def\bs[#1]{\boldsymbol{#1}}
$

$\Kxx, \Kxb, \Kbx, \Kbb, \fx, \fb, \X, \Xb, \lrb[\frac{1}{2}], \bm[1], \bs[\theta]$